# Chap04 - 합성곱 신경망 CNN

**합성곱 신경망(CNN, Convolutional Neural Network)**와 관련된 구성 요소 및 메소드에 대해 알아보고, MNIST 데이터 분류 및 CIFAR-10 데이터에 대해 알아보도록 하자.

## 4.1 CNN 소개

 > 여기서는 CNN에 대해 간략하게 소개한다. 자세히 알고 싶다면 [합성곱신경망](http://excelsior-cjh.tistory.com/79?category=940400)을 참고하면 된다. 

먼저, **합성곱(convolution)**신경망과 **완전연결(fully connected)**신경망의 근본적인 차이점은 계층간의 연결이다.

![](./images/cnn-vs-fcn.png)


완전연결 신경망은 이름에서도 알 수 있듯이 각 유닛(뉴런)이 앞 계층의 모든 유닛과 연결되어 있다. 반면, 합성곱 계층에서는 각각의 유닛은 이전 계층에서 근접해 있는 몇 개의 유닛들에만 연결된다. 또한 모든 유닛은 이전 계층에 동일한 방법으로 연결되어 있으므로 같은 값의 가중치와 구조를 공유한다. 그리고 이 연결 사이에 **합성곱** 연산이 들어 있어 이 신경망을 합성곱 신경망이라 부른다.

<img src="./images/cnn-vs-fcn2.png" width="50%" height="50%">

CNN이 등장한 배경으로는 여러가지 설명이 있는데, 첫 번째는 신경과학적 영감이다. 두 번째는 이미지의 본질에 대한 통찰과 관련이 있고, 세 번째는 학습 이론과 관련이 있다.

보통 합성곱 신경망을 설명할 때 생물학에서 영감을 받은 모델로 설명을 많이 한다. 합성곱 신경망에 대한 블로그 포스팅이나 설명에서 아래의 그림(출처:[distillery.com](https://distillery.com/blog/implementing-human-brain-exploring-potential-convolutional-neural-networks/))과 같은 고양이 그림을 본적이 있을 것이다.

<img src="./images/cnn-motif.png" width="50%" height="50%">

위의 그림은 고양이가 어떤 물체를 인식할 때, 모든 뉴런이 반응하는 것이 아니라 물체의 모양이나 패턴에 따라 특정한 뉴런이 반응한다는 것에 영감을 받아 CNN이 등장하게 되었다는 설명이다. 

두 번째인 공학 관점에서의 설명은, 이미지의 본질에 대해 관한 것이다. 예를 들어서, 이미지 속에서 고양이 얼굴을 찾는다고 하면 이미지 내의 어느 위치에 있는지와 무관하게 찾을 수 있어야 한다. 이것은 동일한 물체가 이미지의 다른 위치에서 발견될 수 있다는 이미지의 속성을 반영한 것이다. 이러한 것을 **불변성(invariance)**이라고 하며, 이러한 불변성은 회전이 발생하거나 조명 조건이 변하더라도 유지되어야 한다. (아래 그림 출처: www.cc.gatech.edu)

<img src="./images/trans_invar.png" width="50%" height="50%">

따라서 객체 인식 시스템을 만들 떄는 변환에 대한 불변성이 있어야 한다. 이러한 관점에서 합성곱 신경망은 전체의 공간 영역에서 이미지의 동일한 특징을 계산한다.

마지막으로, 합성곱 구조는 하나의 정규화(regularization) 과정이라고 볼 수 있다. 정규화(Regularization)은 머신러닝이나 통계학에서 주로 정답의 복잡도에 패널티를 가해 최적화 문제에 제한을 두는 것을 말하며, 이주어진 데이터에 오버피팅(overfitting)을 방지하기위해 사용된다. 합성곱(convolution) 계층은 정해진 크기의 합성곱 보통 매우 작은 합성곱의 크기로 자유도(degree of freedom)를 줄인다.

## 4.2 MNIST 분류기: 버전 2

2장 [2.4 소프트맥스 회귀](http://excelsior-cjh.tistory.com/149)를 이용한 MNIST 분류를 이번에는 CNN으로 MNIST 분류기를 구현해보도록 하자.

### 4.2.1 합성곱(Convolution)
 
합성곱 연산은 합성곱 신경망에서 계층이 연결되는 가장 기본적인 방법이다. 텐서플로에서 기본으로 제공되는 [`conv2d()`](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d)함수를 사용할 수 있다.

```python
tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
```

`x`는 입력 이미지 또는 이전 합성곱 계층에서 출력된 특징맵(Feature Map) 데이터이다. CNN 모델에서는 합성곱 계층(convolution layer)을 층층이 쌓는데 **특징맵(Feature Map)**은 합성곱 계층의 출력을 말한다. 즉, 필터(커널) 및 다른 연산들을 적용한 **'처리된 이미지'**로 생각하면 된다. 필터(또는 커널)은 $m \times n$ 행렬로 구성된 가중치(`W`) 이다. 이러한 필터를 가지고 아래으 그림 처럼 스트라이딩(striding)하며 합성곱 연산을 수행한다. 

![](./images/conv-layer.gif)

합성곱 연산의 결과는 `x`와 `W`의 형태에 따라 달라지며, MNIST 분류기에서는 4차원 값이다. MNIST 이미지 데이터 `x`의 `shape`은 `[None, 28, 28, 1]` 이다. `x`의 `shape`에 대한 설명은 다음과 같다.

- `x = [None, 28, 28, 1]`
    - *None* : 입력될 이미지의 개수는 정해지지 않았으므로 `None`이며, batch-size가 입력된다.
    - *28, 28* : MNIST 데이터는 `28 x 28` 픽셀이다.
    - *1* : 1은 색 채널(channel)을 의미하며, MNIST 데이터는 회색조(grayscale)이미지 이므로 한 개의 채널을 가진다.
    

예제에서 사용될 `W`의 형태는 다음과 같다.

- `W = [5, 5, 1, 32]`
    - *(5, 5, 1)* : 합성곱에 사용될 '윈도우(window)'의 크기를 나타내며, shape는 `5 x 5`이며, `1`은 입력 채널을 의미한다(처음 이미지가 입력될 때는 회색조 이미지이므로 채널이 1이지만, 나중에 합성곱 계층에서 출력된 특징맵의 수를 의미).
    - *32* : 출력될 특징맵의 수(out-channels)다. 합성곱 계층의 아이디어가 이미지 전체에 동일한 특징을 계산하는 것이며, 동일한 특징을 여러번 계산하기 위해 여러개의 필터를 사용하는 것이다.

`strides` 인자는 이미지(또는 특징맵) `x` 위에서 필터 `W`의 이동할 간격을 조절한다. 위의 `strides=[1, 1, 1, 1]`은 필터가 1칸 씩 이동하는 '완전한' 합성곱이다. 스트라이드(Stride)는 보통 1과 같이 작은 값이 더 잘 작동하며, Stride가 1일 경우 입력 데이터의 spatial 크기는 pooling 계층에서만 조절하게 할 수 있다. 

마지막으로 `padding`인자는 `SAME`으로 설정했는데, 패딩(padding)은 합성곱 연산을 수행하기 전, 입력데이터 주변을 특정값으로 채워 늘리는 것을 말한다. `SAME`으로 설정하면 합성곱 계층의 출력결과가 입력 이미지인 `x`의 크기와 같도록 해준다.

#### 활성화 함수

합성곱 계층이나 완전 연결계층에 상관없이 선형 계층에 비선형 함수인 **활성화 함수(activation function)**을 적용하는 것이 일반적이다. 그 이유는 합성곱 연산이나 완전 연결계층의 연산은 선형연산이므로 중간에 비선형 활성화 함수를 사용하지 않으면 신경망 계층이 깊어진다고 해도 아무런 효율이 없기 때문이다.

![](./images/activation.png)

### 4.2.2 풀링(Pooling)

합성곱 계층 다음에는 풀링을 하는 것이 일반적이다. **풀링(pooling)**은 각 특징맵 내에서 집계 함수(평균/최대값)를 사용해 데이터의 크기를 줄이는 것을 의미한다. 

![](./images/pooling.png)

풀링의 배경에는 기술적인 이유와 이론적인 이유가 있다. 기술적 측면에서 풀링은 차례로 처리되는 데이터의 크기를 줄인다. 이 과정으로 모델의 전체 매개변수의 수를 크게 줄일 수 있다. 

풀링의 이론적 측면은 계산된 특징이 이미지 내의 위치에 대한 변화에 영항을 덜 받기 때문이다. 예를 들어 이미지의 우측 상단에서 눈을 찾는 특징은, 눈이 이미지의 중앙에 위치하더라도 크게 영향을 받지 않아야 한다. 그렇기 때문에 풀링을 이용하여 불변성을 찾아내서 공간적 변화를 극복할 수 있다.

<img src="./images/pooling02.png" width="60%" height="60%">

이번 예제에서는 각 특징 맵에 `2 x 2` Max-pooling 연산을 적용한다.

```python
tf.nn.maxpool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
```

`ksize`인자는 풀링의 크기를 정의하고, `strides`인자는 풀링의 이동간격을 의미한다. 이러한 풀링의 결과는 높이(height)와 너비(width)는 절반이 되고 크기는 $\frac{1}{4}$이 된다.

### 4.2.3 드롭아웃

모델에 필요한 마지막 요소는 **드롭아웃(dropout)**이다. 드롭아웃은 정규화(regularization)를 위한 트릭이며 임의의 뉴런을 무작위로 선택에 선택된 뉴런들을 제외하고 학습시키는 방법이다. 

<img src="./images/dropout.png" width="50%" height="50%">

위의 그림처럼 학습 중 값을 `0`으로 세팅해 선택된 뉴런들을 '꺼버리는'방식으로 드롭아웃시킨다. 테스트 단계에서는 드롭아웃 없이 전체 신경망을 그대로 사용한다. 텐서플로에서는 다음과 같이 드롭아웃을 구현할 수 있다.

```python
tf.nn.dropout(layer, keep_prob=0.5)
```

`keep_prob`인자는 각 단계에서 학습을 유지할 뉴런(드롭아웃 시키지 않는)의 비율이다. 나중에 테스트 단계에서는 `keep_prob=1`로 설정한다. 

### 4.2.4 모델

이제 앞에서 알아본 내용을 토대로 CNN모델을 구현해보자. 

먼저, 계층을 만드는데 사용할 헬퍼함수를 정의한다. 이 헬퍼함수를 이용해 짧고 가독성 좋은 코드를 작성할 수 있다. 헬퍼함수는 다음과 같다(`layers.py`)

| 헬퍼함수            | 설명                                                         |
| ------------------- | ------------------------------------------------------------ |
| `weight_variable()` | 합성곱 계층 및 완전 연결 계층의 가중치를 지정한다. 표준편차가 0.1인 절단정규분포를 사용하여 랜덤하게 초기화한다. |
| `bias_variable()`   | 합성곱 계층 및 완전 연결 계층의 편향값을 정의한다. 모두 0.1 상수로 초기화한다. |
| `conv2d()`          | 합성곱 연산을 정의한다. `stride` 를 1로 설정하여 완전한 합성곱을 수행한다. 또한 `padding=SAME` 을 통해 입력과 같은 크기를 출력한다. |
| `max_pool_2x2()`    | 맥스 풀링을 통해 특징 맵의 크기를 $\frac{1}{4}$ 로 줄인다.   |
| `conv_layer()`      | 합성곱 계층으로 `conv2d()` 함수에 정의된 선형 합성곱에 편향값을 더한 후 비선형 함수인 ReLU를 적용한다. |
| `full_layer()`      | 편향을 적용한 완전 연결 계층이다.                            |



In [1]:
# layers.py

import tensorflow as tf

def weight_variables(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

# convolution layer
def conv_layer(input_, shape):
    W = weight_variables(shape)
    b = bias_variable([shape[3]])
    return tf.nn.relu(conv2d(input_, W) + b)

# fully-connected layer
def full_layer(input_, size):
    in_size = int(input_.get_shape()[1])
    W = weight_variables([in_size, size])
    b = bias_variable([size])
    return tf.matmul(input_, W) + b

위의 헬퍼함수를 이용해서 아래의 그림과 같이 CNN모델을 구성해보자.

<img src="./images/cnn-model.png" width="80%" height="80%">

In [4]:
# mnist_cnn.py
import numpy as np
import tensorflow as tf

from layers import conv_layer, max_pool_2x2, full_layer

# Hyper Parameters
STEPS = 5000
MINIBATCH_SIZE = 50

# mnist 불러오기
def mnist_load():
    (train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()

    # Train - Image
    train_x = train_x.astype('float32') / 255
    # Train - Label(OneHot)
    train_y = tf.keras.utils.to_categorical(train_y, num_classes=10)

    # Test - Image
    test_x = test_x.astype('float32') / 255
    # Test - Label(OneHot)
    test_y = tf.keras.utils.to_categorical(test_y, num_classes=10)
    
    return (train_x, train_y), (test_x, test_y)


(train_x, train_y), (test_x, test_y) = mnist_load()

dataset = tf.data.Dataset.from_tensor_slices(({"image": train_x}, train_y))
dataset = dataset.shuffle(1000).repeat().batch(MINIBATCH_SIZE)
iterator = dataset.make_one_shot_iterator()
next_batch = iterator.get_next()

In [11]:
train_y

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [8]:
# CNN 모델링
x = tf.placeholder(tf.float32, shape=[None, 28, 28])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

x_image = tf.reshape(x, [-1, 28, 28, 1])
conv1 = conv_layer(x_image, shape=[5, 5, 1, 32])
conv1_pool = max_pool_2x2(conv1)

conv2 = conv_layer(conv1_pool, shape=[5, 5, 32, 64])
conv2_pool = max_pool_2x2(conv2)

conv2_flat = tf.reshape(conv2_pool, [-1, 7*7*64])
full_1 = tf.nn.relu(full_layer(conv2_flat, 1024))

keep_prob = tf.placeholder(tf.float32)
full_1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob)

y_conv = full_layer(full_1_drop, 10)


# 손실함수
cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_conv, labels=y_))
# 최적화함수
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# 정확도 계산
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # 학습
    for step in range(STEPS):
        batch_xs, batch_ys = sess.run(next_batch)
        _, cost_val = sess.run([train_step, cross_entropy], feed_dict={x: batch_xs['image'], 
                                                                       y_: batch_ys, 
                                                                       keep_prob: 0.5})
        
        if (step+1) % 100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x: batch_xs['image'],
                                                           y_: batch_ys, 
                                                           keep_prob: 1})
            print("Step : {}, cost : {}".format(step+1, cost_val))

Step : 100, cost : 1.3004168272018433
Step : 200, cost : 0.1561235934495926
Step : 300, cost : 0.40108081698417664
Step : 400, cost : 0.36033475399017334
Step : 500, cost : 0.17745564877986908
Step : 600, cost : 0.3791298270225525
Step : 700, cost : 0.13300909101963043
Step : 800, cost : 0.2936430871486664
Step : 900, cost : 0.4453548491001129
Step : 1000, cost : 0.17358259856700897
Step : 1100, cost : 0.4446370303630829
Step : 1200, cost : 0.20072698593139648
Step : 1300, cost : 0.20424240827560425
Step : 1400, cost : 0.34330013394355774


KeyboardInterrupt: 

In [ ]:
X = 